In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from datetime import date
import os

In [ ]:
#需要学习的：
#numpy.ndarray.flatten https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.flatten.html
#numpy flatten的用法：https://blog.csdn.net/qq_18433441/article/details/54916991
#pandas 时间序列分析：https://www.cnblogs.com/foley/p/5582358.html

In [60]:
#1) 设置工作路径,划分数据集
mypath = 'F:\Downloads\lecture02'
os.chdir(mypath)
data = pd.read_csv('Combined_News_DJIA.csv')
data.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [49]:
#划分训练集与测试集
train = data[data['Date'] < '2015-01-01']
test = data[data['Date'] < '2014-12-31']

#选取从第二列到最后一列为X_train,
X_train = train[train.columns[2:]]
#自己定义语料库：将选中的每条新闻 -> 单独的句子,集合在一起，长度为1611*25=40275个句子, 用flatten()把多维数组变为一维，默认按行
corpus = X_train.values.flatten().astype(str)

#【划分X_train】取出训练集, 一共1611行，每行用[xx,xx,xx，..] 25条XX表示里面的新闻内容，array格式
X_train = X_train.values.astype(str)
#对每一行，把25条内容合并为一条, X_train现在每行是由25条新闻组成的一个句子
X_train = np.array([' '.join(x) for x in X_train])

#【划分X_test】
X_test = test[test.columns[2:]]
X_test = X_test.values.astype(str)
X_test = np.array([' '.join(x) for x in X_test])

#【划分y_train】
y_train = train['Label'].values

#【划分y_test】
y_test = test['Label'].values


In [55]:
#把语料corpus（40275句新闻）打印前两个出来看看, 打印前2个X_train出来看看
#corpus[:2]
#X_train[:2]

In [56]:
#2）分词操作： 把句子分词nltk.tokenize， 现在语料库都是单词，X_train中现在25条新闻组成的句子的分词，
from nltk.tokenize import word_tokenize
corpus = [word_tokenize(x) for x in corpus]
X_train = [word_tokenize(x) for x in X_train]
X_test = [word_tokenize(x) for x in X_test]


In [61]:
#打印出来看下
#X_train[:1]
#len(X_train[:1])

In [62]:
#3） 预处理，打包为一个函数
#•小写化
#•删除停止词
#•删除数字与符号
#•lemma

#去除停用词
from nltk.corpus import stopwords
stop = stopwords.words('english')
#去除数字，正则表达式
import re
def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString))
#特殊符号
def isSymbol(inputString):
    return bool(re.match(r'[^\w]', inputString))
#lemma提取词干
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

#定义check函数, 检查每一行中的每个单词是否ok
def check(word):
    if word in stop:
        return False
    elif hasNumbers(word) or isSymbol(word):
        return False
    else:
        return True
    
#定义预处理函数preprocessing
def preprocessing(sen):
    res = []
    for word in sen: #对每一个整体的array数组
        if check(word): #对每一行中的每一个单词，如果check(word)返回True
            #最小化单词，同时去除一些多余的干扰标识
            word = word.lower().replace("b'", '').replace('b"', '').replace('"', '').replace("'", '')
            #进一步提取词干，利用append加入新的数组
            res.append(wordnet_lemmatizer.lemmatize(word))
    return res

In [63]:
# 处理已有的语料，X_train, X_test
corpus = [preprocessing(x) for x in corpus]
X_train = [preprocessing(x) for x in X_train]
X_test = [preprocessing(x) for x in X_test]

In [66]:
#打印出来看看，处理好的数据，格式就是list of list, 
print(X_train[0]) #注意这里的不同，而X_train中是把25条新闻组合在一起后再进行分词（基于每天25条新闻分割）
print(corpus[0])  #corpus是以每条新闻计数的，然后在打散(基于每条新闻分割)

['b', 'georgia', 'two', 'russian', 'warplane', 'country', 'move', 'brink', 'war', 'breaking', 'musharraf', 'impeached', 'russia', 'today', 'column', 'troop', 'roll', 'south', 'ossetia', 'footage', 'fighting', 'youtube', 'russian', 'tank', 'moving', 'towards', 'capital', 'south', 'ossetia', 'reportedly', 'completely', 'destroyed', 'georgian', 'artillery', 'fire', 'b', 'afghan', 'child', 'raped', 'u.n.', 'official', 'say', 'sick', 'three', 'year', 'old', 'raped', 'nothing', 'russian', 'tank', 'entered', 'south', 'ossetia', 'whilst', 'georgia', 'shoot', 'two', 'russian', 'jet', 'b', 'breaking', 'georgia', 'invades', 'south', 'ossetia', 'russia', 'warned', 'would', 'intervene', 'so', 'side', 'b', 'the', 'combatent', 'trial', 'nothing', 'sham', 'salim', 'haman', 'sentenced', 'year', 'kept', 'longer', 'anyway', 'feel', 'like', 'georgian', 'troop', 'retreat', 's.', 'osettain', 'capital', 'presumably', 'leaving', 'several', 'hundred', 'people', 'killed', 'video', 'did', 'u.s.', 'prep', 'georgi

In [71]:
#4)训练NLP模型
#目前我们获得了干净的数据集合,有了人造的语料库，
#先采用Word2Vec，【此处需要了解下参数内容】，相当于转化为词向量

from gensim.models.word2vec import Word2Vec
model = Word2Vec(corpus, size=128, window=5, min_count=5, workers=4)

In [83]:
#打印出来试试, 注意格式问题
model.wv['victory']

array([ 0.11783312,  0.02691937,  0.09941112, -0.45368245,  0.33183682,
        0.32597959, -0.07337862,  0.02844724,  0.03279847,  0.42592683,
       -0.06710878, -0.29577434, -0.15217425,  0.31164819,  0.13869938,
       -0.25164083,  0.15452217, -0.06079166,  0.25306261, -0.12885907,
        0.01831123,  0.13781086, -0.05696175, -0.48023048, -0.04177338,
        0.14853723,  0.06455724,  0.1141708 ,  0.15943082,  0.01353963,
        0.15002282,  0.27767774, -0.45112935,  0.53042918, -0.17026116,
       -0.29705769, -0.2334096 , -0.06159178,  0.25846547, -0.39816603,
        0.18050121, -0.38905147,  0.14459966, -0.16737907,  0.23966882,
       -0.19768161, -0.18540657,  0.19811314,  0.18453735,  0.08856159,
       -0.05768959, -0.05436996,  0.09857335,  0.10677049,  0.14429437,
       -0.00893751,  0.21400209, -0.11511846,  0.27235582, -0.07288903,
       -0.50735539,  0.04730163,  0.04856673, -0.10819522,  0.02564356,
        0.11918142, -0.18032908,  0.14434716,  0.51784837, -0.26

In [96]:
#5)训练NLP模型完成后，用NLP模型表达我们的X
#我们的vec是基于每个单词的，怎么办呢？由于我们文本本身的量很小，我们可以把所有的单词的vector拿过来取个平均值：
#定义一个函数，输入一个任意word_list,得到他们的平均vector值的方法

#先获取所有的vocabulary,注意此处Genism移除了之前的model.vocab,需要替换为model.wv.voccab
vocab = model.wv.vocab
#得到任意text的vector
def get_vector(word_list):
    #新建全为0的array
    res = np.zeros([128])
    count=0
    for word in word_list:
        res += model[word]
        count += 1
    return res/count

In [97]:
#这样，我们可以同步把我们的X都给转化成128维的一个vector list
get_vector(['hello', 'from', 'the', 'other', 'side'])

D:\Software\annacoda 5.01\installation\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


array([  1.35594710e-01,   4.78771497e-03,   2.08560316e-02,
        -3.06096678e-01,   3.00854318e-01,   2.74248731e-01,
        -9.45481402e-02,   3.24749298e-02,   2.60592854e-02,
         2.72211928e-01,  -5.20233473e-02,  -2.49901214e-01,
        -1.35818139e-01,   2.30652633e-01,   1.17855432e-01,
        -1.58467300e-01,   1.31108695e-01,  -5.22524355e-02,
         1.56251449e-01,  -1.56735947e-01,   4.10375485e-02,
         7.54008744e-02,  -5.78824667e-02,  -4.36764856e-01,
        -5.68012707e-05,   1.66545488e-01,   9.81779173e-02,
         1.13883892e-01,   1.24993741e-01,  -2.06849872e-02,
         5.01941387e-02,   1.95990811e-01,  -3.10149046e-01,
         4.19034262e-01,  -1.97779381e-01,  -2.29321394e-01,
        -1.60880928e-01,  -5.14058918e-03,   2.68698773e-01,
        -4.23572254e-01,   1.37644243e-01,  -4.04576414e-01,
         1.73564543e-01,  -7.31670639e-02,   1.70787470e-01,
        -1.12505922e-01,  -5.55463632e-02,   1.32595204e-01,
         2.16302520e-01,

In [98]:
wordlist_train = X_train
wordlist_test = X_test

#KeyError: "word 'impeached' not in vocabulary" 出现错误，这里转换失败
X_train = [get_vector(x) for x in X_train] 
X_test = [get_vector(x) for x in X_test]

print(X_train[10])

D:\Software\annacoda 5.01\installation\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


KeyError: "word 'impeached' not in vocabulary"

In [101]:
#6) 建立ML模型
#这里，因为我们128维的每一个值都是连续关系的。不是分裂开考虑的。所以，道理上讲，我们是不太适合用RandomForest这类把每个column当做单独的variable来看的方法。（当然，事实是，你也可以这么用）
#我们来看看比较适合连续函数的方法：SVM

from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score

params = [0.1,0.5,1,3,5,7,10,12,16,20,25,30,35,40]
test_scores = []
for param in params:
    clf = SVR(gamma = param)
    test_score = cross_val_score(clf, X_train, y_train, cv=3, scoring='roc_auc')
    test_scores.append(np.mean(test_score))

#画图结果
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(params, test_scores)
plt.title("Param vs CV AUC Score");

#后期使用CNN方法


ValueError: setting an array element with a sequence.